# Adding records

The `addRecord()` method creates a JSON string with information about your Senzing version.

In [ ]:
import os
import sys
import json
import pandas as pd
import qgrid

## G2Engine

The G2Engine API...

In [ ]:
from G2Engine import G2Engine

## Initialize variables

Create variables used in `addRecord()`.

In [ ]:
module_name = 'pyG2EngineForAddRecord'
senzing_directory = os.environ.get("SENZING_DIR", "/opt/senzing")
senzing_python_directory = "{0}/g2/python".format(senzing_directory)
g2module_ini_pathname = "{0}/G2Module.ini".format(senzing_python_directory)
verbose_logging = False
config_id = bytearray([])

## System path

Update system path.

In [ ]:
sys.path.append(senzing_python_directory)

## Initialization

To start using Senzing G2Engine, create and initialize an instance.
This should be done once per process.
The `init()` method accepts the following parameters:

- **module_name:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **g2module_ini_pathname:** A fully qualified path to the G2 engine INI file (often /opt/senzing/g2/python/G2Module.ini)
- **verbose_logging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)
- **config_id:** (optional) The identifier value for the engine configuration can be returned here.

Calling this function will return "0" upon success.

In [ ]:
g2_engine = G2Engine()
result = g2_engine.init(module_name, g2module_ini_pathname, verbose_logging, config_id)
print(result)

## Prime Engine

The `primeEngine()` method may optionally be called to pre-initialize some of the heavier weight internal resources of the G2 engine.

In [ ]:
response = g2_engine.primeEngine()

## addRecord()

Once the Senzing engine is initialized, use addRecord() to load a record into the Senzing repository -- addRecord() can be called as many times as desired and from multiple threads at the same time. The addRecord() function returns "0" upon success, and accepts four parameters as input:

- **datasource_code:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID, used to identify distinct records
- **data_string:** A JSON document with the attribute data for the record
- **load_id:** The observation load ID for the record; value can be null and will default to data_source


In [ ]:
datasource_code = "TEST"
record_id = "1"
load_id = None
names = {
    "NAME_TYPE": "PRIMARY",
    "NAME_LAST": "Smith",
    "NAME_FIRST": "John",
    "NAME_MIDDLE": "M"    
}
data = {
    "NAMES": [names],
    "PASSPORT_NUMBER": "PP11111",
    "PASSPORT_COUNTRY": "US",
    "DRIVERS_LICENSE_NUMBER": "DL11111",
    "SSN_NUMBER": "111-11-1111"
}

In [ ]:
names_df = pd.DataFrame(data['NAMES'], index=[0])
data['NAMES'] = [names_df]
data_df = pd.DataFrame(data)

In [ ]:
data_df

### QGrid table
Despite allowing multiple rows, just the first is being used.

In [ ]:
widget_qgrid = qgrid.show_grid(data_df, show_toolbar=True)
widget_qgrid

In [ ]:
changed_data_df = widget_qgrid.get_changed_df()
all_entries_json = json.loads(changed_data_df.to_json(orient='index'))

### Convert DataFrame to JSON strings

In [ ]:
def data_frame_to_senzing_data(data_frame):
    result = []
    for row in data_frame.iterrows():
        entry = {}
        for col_value in row[1].iteritems():
            value = col_value[1]
            if isinstance(value, pd.DataFrame):
                value = data_frame_to_senzing_data(value)
            entry[col_value[0]] = value
        result.append(entry)
    return result

### addRecord()

In [ ]:
for data in data_frame_to_senzing_data(changed_data_df):
    data_string = json.dumps(data)
    result = g2_engine.addRecord(datasource_code, record_id, data_string, load_id)
    print(result)

## getRecord()

Use `getRecord()` to retrieve a single record from the data repository; the record is assigned in JSON form to a user-designated buffer, and the function itself returns "0" upon success. Once the Senzing engine is initialized, `getRecord()` can be called as many times as desired and from multiple threads at the same time. The `getRecord()` function accepts the following parameters as input:

- **data_source:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID, used to identify the record for retrieval
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here

In [ ]:
response_list = []
result = g2_engine.getRecord(datasource_code, record_id, response_list)

response_string = "".join(response_list)
response_dictionary = json.loads(response_string)
response = json.dumps(response_dictionary, sort_keys=True, indent=4)
print("Result: {0}\n{1}".format(result, response))

## getEntityByRecordID()

Use `getEntityByRecordID()` to retrieve entity data based on the record ID of a particular data record. This function accepts the following parameters as input:

- **datasource_code:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID for a particular data record
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
response_list = []
result = g2_engine.getEntityByRecordID(datasource_code, record_id, response_list)

response_string = "".join(response_list)
response_dictionary = json.loads(response_string)
response = json.dumps(response_dictionary, sort_keys=True, indent=4)
print("Result: {0}\n{1}".format(result, response))

## getEntityByEntityID()

Use `getEntityByEntityID()` to retrieve entity data based on the ID of a resolved identity. This function accepts the following parameters as input:

- **entity_id:** The numeric ID of a resolved entity
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
entity_id = 1
response_list = []
result = g2_engine.getEntityByEntityID(entity_id, response_list)

response_string = "".join(response_list)
response_dictionary = json.loads(response_string)
response = json.dumps(response_dictionary, sort_keys=True, indent=4)
print("Result: {0}\n{1}".format(result, response))


## searchByAttributes()

Use `searchByAttributes()` to retrieve entity data based on a user-specified set of entity attributes. This function accepts the following parameters as input:

- **data_string:** A JSON document with the attribute data to search for
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
response_list = []
result = g2_engine.searchByAttributes(data_string, response_list)

response_string = "".join(response_list)
response_dictionary = json.loads(response_string)
response = json.dumps(response_dictionary, sort_keys=True, indent=4)
print("Result: {0}\n{1}".format(result, response))